# Tarea Covid

Recrear, simular datos de la pandemia en Costa Rica, durante el año 2020, 365 reportes: un por cada día del año

Consideraciones:
Cada habitante de Costa Rica que puede vivir en cualquier lugar, sea:
7 provincias
82 cantones
xxx distritos

$ crear aleatoriamente reportes de datos de infección diarios que se requiere: 

crear un dataframe /reporte de nuevos contagios

# cedula , provincia, canton, distrito, fecha resultado positivo, fecha resultado negativo , estado [subceptible, infectado, recuperado, fallecido]

# se puede volver a contagiar : si . No se puede contagiar si está contagiodo
# se puede fallecer : si

Intregable:
365 archivos del tipo CSV que representa el reporte diario de sólo datos actualizados (importante)


descricion de de columnas: 

cedula: x-xxxx-xxxx
provincia [san jose, alajuela, cartago, heredia, guanacaste, puntarenas, limon]
cantones: los 82
distritos: los reales (todos los que hay)
fechas: año-mes-dia
estado: categorical [subceptible, infectado, recuperado, fallecido]

cantidas de datos:
cantidad. aleatoria de personas. uniforme rango [0-1000]
ubicacion. aleatorio. con sentido real (san jose. tibas, llorente)

fecha de resultado negativo (número de días por curarse): 14 días. aleatorio normal seagado a la derecha en 14 días

fallecer: distribución uniforme [si/no]


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta

In [2]:
## Origen de datos: TSE
#location = pd.read_csv('data/Distelec.txt', header=None engine = 'python-fwf')
location = pd.read_csv('data/Distelec.txt', header=None, encoding='ISO8859-1')
#converters={3: str.lstrip}
location.columns = ['ID_UBICACION', 'PROVINCIA', 'DISTRITO', 'CANTON']
location.head()

,ID_UBICACION,PROVINCIA,DISTRITO,CANTON
0,101001,SAN JOSE,CENTRAL,HOSPITAL ...
1,101002,SAN JOSE,CENTRAL,ZAPOTE ...
2,101003,SAN JOSE,CENTRAL,SAN FRANCISCO DE DOS RIOS ...
3,101004,SAN JOSE,CENTRAL,URUCA ...
4,101005,SAN JOSE,CENTRAL,MATA REDONDA ...


In [3]:
location['DISTRITO'] = location.loc[: , 'DISTRITO'].str.strip()

In [4]:
location.iloc[0]

ID_UBICACION                                               101001
PROVINCIA                                                SAN JOSE
DISTRITO                                                  CENTRAL
CANTON          HOSPITAL                                      ...
Name: 0, dtype: object

In [5]:
## Origen de datos: TSE
persons = pd.read_csv('data/PADRON_COMPLETO.txt', header=None, encoding='ISO8859-1')
persons.columns = ['CEDULA', 'CODELEC', 'SEXO', 'FEC_VENCIMIENTO', 'JUNTA', 'NOMBRE', '1ER_APELLIDO', '2DO_APELLIDO']
persons.head()

,CEDULA,CODELEC,SEXO,FEC_VENCIMIENTO,JUNTA,NOMBRE,1ER_APELLIDO,2DO_APELLIDO
0,100339724,109007,1,20231119,0,JOSE,DELGADO,CORRALES
1,100842598,108001,2,20261024,0,CARMEN,CORRALES,MORALES
2,100842638,301027,1,20220531,0,RAFAEL,AGUERO,MORA
3,100964116,104022,1,20280207,0,MANUEL,DELGADO,GUZMAN
4,100986470,205012,2,20220630,0,MARIA,SALAS,CHAVARRIA


In [6]:
### Unir data de DISTELEC y PADRON
data = persons.merge(location, left_on='CODELEC', right_on='ID_UBICACION')
data.head()

,CEDULA,CODELEC,SEXO,FEC_VENCIMIENTO,JUNTA,NOMBRE,1ER_APELLIDO,2DO_APELLIDO,ID_UBICACION,PROVINCIA,DISTRITO,CANTON
0,100339724,109007,1,20231119,0,JOSE,DELGADO,CORRALES,109007,SAN JOSE,SANTA ANA,PIEDADES ...
1,101300341,109007,2,20220228,0,ERMIDA,SOLIS,FALLAS,109007,SAN JOSE,SANTA ANA,PIEDADES ...
2,101510294,109007,2,20260406,0,BERTA,RAMIREZ,GUILLEN,109007,SAN JOSE,SANTA ANA,PIEDADES ...
3,101610291,109007,2,20250710,0,ADABERTA,CORDERO,JIMENEZ,109007,SAN JOSE,SANTA ANA,PIEDADES ...
4,101670103,109007,1,20280207,0,RAFAEL,CARMONA,UMAÑA,109007,SAN JOSE,SANTA ANA,PIEDADES ...


In [7]:
## Eliminar data no requerida y agregar columnas requeridas
data = data.drop(data.columns[[1, 3, 4, 8]], axis=1)  # df.columns is zero-based pd.Index
data.dtypes

CEDULA           int64
SEXO             int64
NOMBRE          object
1ER_APELLIDO    object
2DO_APELLIDO    object
PROVINCIA       object
DISTRITO        object
CANTON          object
dtype: object

In [8]:
## Añadir columnas requeridas (Estado['Susceptible', 'Contagiado', 'Recuperado', 'Fallecido'], Fecha resultado positivo, Fecha resultado negativo, Fecha fallecido)
data = data.assign(ESTADO='Susceptible', FECHA_RESULTADO_POSITIVO=np.nan, FECHA_RESULTADO_NEGATIVO=np.nan, FECHA_FALLECIMIENTO=np.nan)
#data['CEDULA'] = pd.Categorical(data.CEDULA)
data['ESTADO'] = pd.Categorical(data.ESTADO)
data['FECHA_RESULTADO_POSITIVO'] = pd.to_datetime(data['FECHA_RESULTADO_POSITIVO'])
data['FECHA_RESULTADO_NEGATIVO'] = pd.to_datetime(data['FECHA_RESULTADO_NEGATIVO'])
data['FECHA_FALLECIMIENTO'] = pd.to_datetime(data['FECHA_FALLECIMIENTO'])
sexo_dic = {1: 'Masculino', 2: 'Femenino'}
data['SEXO'] = data['SEXO'].map(sexo_dic)
data['SEXO'] = pd.Categorical(data.SEXO)
data.dtypes

CEDULA                               int64
SEXO                              category
NOMBRE                              object
1ER_APELLIDO                        object
2DO_APELLIDO                        object
PROVINCIA                           object
DISTRITO                            object
CANTON                              object
ESTADO                            category
FECHA_RESULTADO_POSITIVO    datetime64[ns]
FECHA_RESULTADO_NEGATIVO    datetime64[ns]
FECHA_FALLECIMIENTO         datetime64[ns]
dtype: object

In [9]:
data.describe()

,CEDULA
count,3.469004e+06
mean,3.136486e+08
std,2.294382e+08
min,1.003397e+08
25%,1.130509e+08
50%,2.062905e+08
75%,5.029701e+08
max,9.014408e+08


In [10]:
data.head()

,CEDULA,SEXO,NOMBRE,1ER_APELLIDO,2DO_APELLIDO,PROVINCIA,DISTRITO,CANTON,ESTADO,FECHA_RESULTADO_POSITIVO,FECHA_RESULTADO_NEGATIVO,FECHA_FALLECIMIENTO
0,100339724,Masculino,JOSE,DELGADO,CORRALES,SAN JOSE,SANTA ANA,PIEDADES ...,Susceptible,NaT,NaT,NaT
1,101300341,Femenino,ERMIDA,SOLIS,FALLAS,SAN JOSE,SANTA ANA,PIEDADES ...,Susceptible,NaT,NaT,NaT
2,101510294,Femenino,BERTA,RAMIREZ,GUILLEN,SAN JOSE,SANTA ANA,PIEDADES ...,Susceptible,NaT,NaT,NaT
3,101610291,Femenino,ADABERTA,CORDERO,JIMENEZ,SAN JOSE,SANTA ANA,PIEDADES ...,Susceptible,NaT,NaT,NaT
4,101670103,Masculino,RAFAEL,CARMONA,UMAÑA,SAN JOSE,SANTA ANA,PIEDADES ...,Susceptible,NaT,NaT,NaT


In [11]:
data.query('CEDULA == "206940116"')

,CEDULA,SEXO,NOMBRE,1ER_APELLIDO,2DO_APELLIDO,PROVINCIA,DISTRITO,CANTON,ESTADO,FECHA_RESULTADO_POSITIVO,FECHA_RESULTADO_NEGATIVO,FECHA_FALLECIMIENTO
3106759,206940116,Masculino,DIEGO GERARDO,ARCE,ESPINOZA,ALAJUELA,GRECIA,PUENTE DE PIEDRA ...,Susceptible,NaT,NaT,NaT


In [12]:
## Obtener dataframe de recuperados utilizando un sesgo de 14 días a la derecha
def get_recovered_people(current_day, df):
    #sick_duration = np.arange(start = 7, stop = 22)
    return df.loc[df.FECHA_RESULTADO_NEGATIVO == current_day]

In [15]:
## Obtener dataframe de contagiados utilizando un aleatorio entre 0-1000 para cada día
sick_duration = np.arange(start = 7, stop = 22)
def get_random_recovery_day(current_day):
    probability = [.02, .02, .03, .03, .03, .03, .03, .5, .06, .06, .04, .04, .04, .04, .03]
    random_days = np.random.choice(a = sick_duration, p = probability, size = 1)[0].astype(np.float)
    return current_day+timedelta(days=random_days)

def get_sick_people(current_day, df):
    x = np.random.randint(1000)
    result = df.sample(n = x)
    result['FECHA_RESULTADO_POSITIVO'].fillna(current_day, inplace = True)
    #result['FECHA_RESULTADO_NEGATIVO'].fillna(current_day+timedelta(days=get_random_recovery_day()), inplace = True)
    result['FECHA_RESULTADO_NEGATIVO'] = result['FECHA_RESULTADO_NEGATIVO'].apply(lambda a: get_random_recovery_day(current_day), axis=1)
    # set random weight based on parameters
    #result['FECHA_RESULTADO_NEGATIVO'] = df.apply(
    #    lambda x: current_day+timedelta(days=get_random_recovery_day()), axis = 1
    #)
    return result

In [16]:
## Obtener dataframe de muertos utilizando una mortalidad del 0.7% aprox de los contagidos
def get_dead_people(current_day, df):
    result = df.sample(frac = 0.007)
    result['FECHA_FALLECIMIENTO'].fillna(current_day, inplace = True)
    return result

In [17]:
# Iterar entre los 365 dias del año
start_date = date(2020, 1, 1)
end_date = date(2020, 2, 1)
daterange = pd.date_range(start_date, end_date)

# Crear dataframes en blanco
total_dead_people = pd.DataFrame(data=None, columns=data.columns)
total_sick_people = pd.DataFrame(data=None, columns=data.columns)
total_recovered_people = pd.DataFrame(data=None, columns=data.columns)

# Generar reportes de contagiados, muertes y recuperados
for day in daterange:
    print ("Calculando datos para:" + day.strftime("%Y-%m-%d"))

    ## Obtener cantidad de enfermos del dia
    sick_people_df = get_sick_people(day, data)
    total_sick_people = total_sick_people.append(sick_people_df, ignore_index=True)
    data.drop(sick_people_df.index)

    ## Obtener recuperados y volver a unir al data
    recovered_people_df = get_recovered_people(day, total_sick_people)
    total_recovered_people = total_recovered_people.append(recovered_people_df, ignore_index=True)
    data.append(recovered_people_df, ignore_index=True)

    ## Obtener cantidad de muertos del total de enfermos
    dead_people_df = get_dead_people(day, total_sick_people)
    total_sick_people.drop(dead_people_df.index)

    day_resume = sick_people_df.copy()
    day_resume = day_resume.append(recovered_people_df, ignore_index=True)
    day_resume = day_resume.append(dead_people_df, ignore_index=True)

    day_resume.to_csv('results/resume' + day.strftime("%Y-%m-%d") + '.csv', index = True)
    total_sick_people.to_csv('results/total_sick_people' + day.strftime("%Y-%m-%d") + '.csv', index = True)

#data.append(total_sick_people, ignore_index=True)
#data.append(total_sick_people, ignore_index=True)

Calculando datos para:2020-01-01


TypeError: <lambda>() got an unexpected keyword argument 'axis'

## Referencias
https://www.bbc.com/mundo/noticias-51708029